In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ecommerce-text-classification/ecommerceDataset.csv


In [8]:
df = pd.read_csv("/kaggle/input/ecommerce-text-classification/ecommerceDataset.csv",names=['Category','Description'])

In [9]:
import fasttext

In [10]:
print(df.shape)
df.head(3)

(50425, 2)


,Category,Description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...


In [11]:
df

,Category,Description
0,Household,Paper Plane Design Framed Wall Hanging Motivat...
1,Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,Household,Incredible Gifts India Wooden Happy Birthday U...
...,...,...
50420,Electronics,Strontium MicroSD Class 10 8GB Memory Card (Bl...
50421,Electronics,CrossBeats Wave Waterproof Bluetooth Wireless ...
50422,Electronics,Karbonn Titanium Wind W4 (White) Karbonn Titan...
50423,Electronics,"Samsung Guru FM Plus (SM-B110E/D, Black) Colou..."


In [12]:
df.isnull().sum()

Category       0
Description    1
dtype: int64

In [13]:
df.dropna(inplace=True)
df.shape

(50424, 2)

In [14]:
df.Category.unique()

array(['Household', 'Books', 'Clothing & Accessories', 'Electronics'],
      dtype=object)

In [15]:
df.Category.replace("Clothing & Accessories", "Clothing_Accessories", inplace=True)

# When you train a fasttext model, it expects labels to be specified with label prefix. We will just create a third column in the dataframe that has label as well as the product description

In [16]:
df['Category'] = '__label__' + df['Category'].astype(str)
df.head(5)

,Category,Description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ..."
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1..."
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...


In [17]:
df['category_description'] = df['Category'] + ' ' + df['Description']
df.head(3)

,Category,Description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__Household Paper Plane Design Framed W...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__Household SAF 'Floral' Framed Paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__Household SAF 'UV Textured Modern Art...


# Pre-procesing

1. Remove punctuation
2. Remove extra space
3. Make the entire sentence lower case

In [26]:
import re

def preprocess(text):
    text = re.sub(r'[^\w\s\']',' ', text)
    text = re.sub(' +', ' ', text)
    return text.strip().lower()

In [27]:
df['category_description'].map(preprocess)[0]

'__label__household paper plane design framed wall hanging motivational office decor art prints 8 7 x 8 7 inch set of 4 painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it this is an special series of paintings which makes your wall very beautiful and gives a royal touch this painting is ready to hang you would be proud to possess this unique painting that is a niche apart we use only the most modern and efficient printing technology on our prints with only the and inks and precision epson roland and hp printers this innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime we print solely with top notch 100 inks to achieve brilliant and true colours due to their high level of uv resistance our prints retain their beautiful colours for many years add colour and style to your living space with this digitally printed painting some are for pleasure and some for eternal bli

In [28]:
df['Description'][0]

'Paper Plane Design Framed Wall Hanging Motivational Office Decor Art Prints (8.7 X 8.7 inch) - Set of 4 Painting made up in synthetic frame with uv textured print which gives multi effects and attracts towards it. This is an special series of paintings which makes your wall very beautiful and gives a royal touch. This painting is ready to hang, you would be proud to possess this unique painting that is a niche apart. We use only the most modern and efficient printing technology on our prints, with only the and inks and precision epson, roland and hp printers. This innovative hd printing technique results in durable and spectacular looking prints of the highest that last a lifetime. We print solely with top-notch 100% inks, to achieve brilliant and true colours. Due to their high level of uv resistance, our prints retain their beautiful colours for many years. Add colour and style to your living space with this digitally printed painting. Some are for pleasure and some for eternal blis

In [30]:
df['category_description'] = df['category_description'].map(preprocess)
df.head()

,Category,Description,category_description
0,__label__Household,Paper Plane Design Framed Wall Hanging Motivat...,__label__household paper plane design framed w...
1,__label__Household,"SAF 'Floral' Framed Painting (Wood, 30 inch x ...",__label__household saf 'floral' framed paintin...
2,__label__Household,SAF 'UV Textured Modern Art Print Framed' Pain...,__label__household saf 'uv textured modern art...
3,__label__Household,"SAF Flower Print Framed Painting (Synthetic, 1...",__label__household saf flower print framed pai...
4,__label__Household,Incredible Gifts India Wooden Happy Birthday U...,__label__household incredible gifts india wood...


In [31]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)

In [32]:
train.shape, test.shape

((40339, 3), (10085, 3))

In [33]:
train.to_csv("ecommerce.train", columns=["category_description"], index=False, header=False)
test.to_csv("ecommerce.test", columns=["category_description"], index=False, header=False)

In [34]:
import fasttext

model = fasttext.train_supervised(input="ecommerce.train")
model.test("ecommerce.test")

Read 4M words
Number of words:  79333
Number of labels: 4
Progress: 100.0% words/sec/thread: 1630679 lr:  0.000000 avg.loss:  0.188594 ETA:   0h 0m 0s


(10084, 0.9690598968663229, 0.9690598968663229)

First parameter (10084) is test size. Second and third parameters are precision and recall respectively. You can see we are getting around 96% precision which is pretty good

In [35]:
# Now let's do prediction for few product descriptions

In [36]:
model.predict("wintech assemble desktop pc cpu 500 gb sata hdd 4 gb ram intel c2d processor 3")

(('__label__electronics',), array([0.99893457]))

In [37]:
model.predict("think and grow rich deluxe edition")

(('__label__books',), array([1.00000989]))

In [38]:
model.get_nearest_neighbors("painting")

[(0.9988144040107727, 'globalepartner'),
 (0.9988097548484802, 'bxfs7751in'),
 (0.998807966709137, 'childproof'),
 (0.9988042712211609, 'burma'),
 (0.9987978339195251, '3cmx13cmx2'),
 (0.9987949132919312, 'dhokla'),
 (0.9987917542457581, 'aprox'),
 (0.9987878203392029, 'dripcafe'),
 (0.9987829327583313, 'horseman'),
 (0.9987812638282776, '448mm')]

In [39]:
model.get_nearest_neighbors("sony")

[(0.9992974996566772, 'adversary'),
 (0.9992974996566772, '540mb'),
 (0.9992974996566772, 'ssdsand'),
 (0.9992974996566772, '2800mb'),
 (0.9992974996566772, '3400mb'),
 (0.9992972612380981, 'synchronously'),
 (0.9992943406105042, 'unipivot'),
 (0.9992943406105042, '83v'),
 (0.9992943406105042, '20mx'),
 (0.9992943406105042, 'bandpass')]

In [40]:
model.get_nearest_neighbors("banglore")

[(0.0, 'to'),
 (0.0, 'and'),
 (0.0, 'a'),
 (0.0, 'with'),
 (0.0, 'for'),
 (0.0, 'is'),
 (0.0, '</s>'),
 (0.0, 'jamesmtabor'),
 (0.0, 'mc10'),
 (0.0, 'udaipurcraft')]